In [1]:
import gensim
import os
import json
import numpy as np

## Load and Processing Training Dataset

In [2]:
dataset_file = "UD_English-EWT/en_ewt-ud-train.conllu"
lines = open(dataset_file, "r").readlines()

sentence_word = []
sentences_word = []

sentence_head_id = []
sentences_head_id = []

sentence_pos = []
sentences_pos = []

sentence_deprel = []
sentences_deprel = []

for line in lines:
    l = line.split()
    if len(l) > 0 and (l[0].isdigit()):
        sentence_word.append(l[1])
        sentence_head_id.append(int(l[6]))
        sentence_pos.append(l[3])
        sentence_deprel.append(l[7])
    if len(l) == 0:
        sentences_word.append(sentence_word)
        sentence_word = []
        sentences_head_id.append(sentence_head_id)
        sentence_head_id = []
        sentences_pos.append(sentence_pos)
        sentence_pos = []
        sentences_deprel.append(sentence_deprel)
        sentence_deprel = []
    
# sentences_word
# sentences_head_id
# sentences_pos[0]
# sentences_deprel[0]

def check_no_child(sta, buff, iden, sentence_h_id):
    for i in sta:
        if (sentence_h_id[i] == iden):
            return False
    for i in buff:
        if (sentence_h_id[i] == iden):
            return False
    return True

def rdep(l, ind, tree):
    try:
        e = l[ind]
    except:
        return None
    for edge in tree:
        if edge[0] == e:
            return edge[2]
    return None

def ldep(l, ind, tree):
    try:
        e = l[ind]
    except:
        return None
    for edge in tree:
        if edge[2] == e:
            return edge[0]
    return None

In [3]:
filename = "configurations_training.txt"

try:
    os.remove(filename)
except OSError:
    pass

f = open(filename, "a")

for i in range(len(sentences_word)):
#     print(sentences_word[i], sentences_head_id[i])
    sentence_word = list(sentences_word[i])
    sentence_word.insert(0, 'ROOT')
    sentence_head_id = list(sentences_head_id[i])
    sentence_head_id.insert(0, -1)
    sentence_pos = list(sentences_pos[i])
    sentence_pos.insert(0, 'ROOT-POS')
    sentence_deprel = list(sentences_deprel[i])
    sentence_deprel.insert(0, 'RROOT')
    

    stack = [0] # 'ROOT' -> 0
    buffer = list(range(1, len(sentences_word[i])+1)) # indices of all words
    tree = []
    
    
    conf = [\
            sentence_word[stack[-1]] if len(stack)>0 else None, \
            sentence_pos[stack[-1]] if len(stack)>0 else None, \
            sentence_deprel[stack[-1]] if len(stack)>0 else None, \
            sentence_word[buffer[0]] if len(buffer)>0 else None, \
            sentence_pos[buffer[0]] if len(buffer)>0 else None, \
            sentence_deprel[buffer[0]] if len(buffer)>0 else None, \
            sentence_word[buffer[1]] if len(buffer)>1 else None, \
            sentence_pos[buffer[1]] if len(buffer)>1 else None, \
            sentence_deprel[buffer[1]] if len(buffer)>1 else None, \
            sentence_word[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
            sentence_pos[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
            sentence_deprel[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
            sentence_word[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
            sentence_pos[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
            sentence_deprel[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
            sentence_word[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
            sentence_pos[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
            sentence_deprel[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
            sentence_word[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
            sentence_pos[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
            sentence_deprel[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
           ]
        
    f.write(json.dumps(conf))
    
    
    while len(buffer) > 0:
        head_id_stack_right = sentence_head_id[stack[-1]] if len(stack) > 0 else -2
        head_id_buffer_left = sentence_head_id[buffer[0]]
        
        if (head_id_stack_right == buffer[0]):
            tree.append((stack[-1], sentence_deprel[stack[-1]], buffer[0])) # add stack[-1] to tree
            stack.pop(-1) # left arc
            f.write(" <> 0\n")
        
        elif (len(stack) > 0 and head_id_buffer_left == stack[-1] and \
              check_no_child(stack, buffer, buffer[0], sentence_head_id)):
            tree.append((buffer[0], sentence_deprel[buffer[0]], stack[-1])) # add buffer[0] to tree
            buffer[0] = stack.pop(-1) # right arc
            f.write(" <> 1\n")
            
        else:
            stack.append(buffer.pop(0)) # shift
            f.write(" <> 2\n")
        
        conf = [\
                sentence_word[stack[-1]] if len(stack)>0 else None, \
                sentence_pos[stack[-1]] if len(stack)>0 else None, \
                sentence_deprel[stack[-1]] if len(stack)>0 else None, \
                sentence_word[buffer[0]] if len(buffer)>0 else None, \
                sentence_pos[buffer[0]] if len(buffer)>0 else None, \
                sentence_deprel[buffer[0]] if len(buffer)>0 else None, \
                sentence_word[buffer[1]] if len(buffer)>1 else None, \
                sentence_pos[buffer[1]] if len(buffer)>1 else None, \
                sentence_deprel[buffer[1]] if len(buffer)>1 else None, \
                sentence_word[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
                sentence_pos[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
                sentence_deprel[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
                sentence_word[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
                sentence_pos[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
                sentence_deprel[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
                sentence_word[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
                sentence_pos[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
                sentence_deprel[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
                sentence_word[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
                sentence_pos[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
                sentence_deprel[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
               ]
        
        f.write(json.dumps(conf))
    f.write(" <> -1\n")
#     print(i)
print("Wrote ", filename)

Wrote  configurations_training.txt


## Loading and Processing Test Dataset

In [4]:
dataset_file = "UD_English-EWT/en_ewt-ud-test.conllu"
lines = open(dataset_file, "r").readlines()

sentence_word = []
sentences_word = []

sentence_head_id = []
sentences_head_id = []

sentence_pos = []
sentences_pos = []

sentence_deprel = []
sentences_deprel = []

for line in lines:
    l = line.split()
    if len(l) > 0 and (l[0].isdigit()):
        sentence_word.append(l[1])
        sentence_head_id.append(int(l[6]))
        sentence_pos.append(l[3])
        sentence_deprel.append(l[7])
    if len(l) == 0:
        sentences_word.append(sentence_word)
        sentence_word = []
        sentences_head_id.append(sentence_head_id)
        sentence_head_id = []
        sentences_pos.append(sentence_pos)
        sentence_pos = []
        sentences_deprel.append(sentence_deprel)
        sentence_deprel = []
    
# sentences_word
# sentences_head_id
# sentences_pos[0]
# sentences_deprel[0]

def check_no_child(sta, buff, iden, sentence_h_id):
    for i in sta:
        if (sentence_h_id[i] == iden):
            return False
    for i in buff:
        if (sentence_h_id[i] == iden):
            return False
    return True

def rdep(l, ind, tree):
    try:
        e = l[ind]
    except:
        return None
    for edge in tree:
        if edge[0] == e:
            return edge[2]
    return None

def ldep(l, ind, tree):
    try:
        e = l[ind]
    except:
        return None
    for edge in tree:
        if edge[2] == e:
            return edge[0]
    return None

In [5]:
filename = "configurations_testing.txt"

try:
    os.remove(filename)
except OSError:
    pass

f = open(filename, "a")

for i in range(len(sentences_word)):
#     print(sentences_word[i], sentences_head_id[i])
    sentence_word = list(sentences_word[i])
    sentence_word.insert(0, 'ROOT')
    sentence_head_id = list(sentences_head_id[i])
    sentence_head_id.insert(0, -1)
    sentence_pos = list(sentences_pos[i])
    sentence_pos.insert(0, 'ROOT-POS')
    sentence_deprel = list(sentences_deprel[i])
    sentence_deprel.insert(0, 'RROOT')
    

    stack = [0] # 'ROOT' -> 0
    buffer = list(range(1, len(sentences_word[i])+1)) # indices of all words
    tree = []
    
    
    conf = [\
            sentence_word[stack[-1]] if len(stack)>0 else None, \
            sentence_pos[stack[-1]] if len(stack)>0 else None, \
            sentence_deprel[stack[-1]] if len(stack)>0 else None, \
            sentence_word[buffer[0]] if len(buffer)>0 else None, \
            sentence_pos[buffer[0]] if len(buffer)>0 else None, \
            sentence_deprel[buffer[0]] if len(buffer)>0 else None, \
            sentence_word[buffer[1]] if len(buffer)>1 else None, \
            sentence_pos[buffer[1]] if len(buffer)>1 else None, \
            sentence_deprel[buffer[1]] if len(buffer)>1 else None, \
            sentence_word[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
            sentence_pos[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
            sentence_deprel[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
            sentence_word[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
            sentence_pos[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
            sentence_deprel[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
            sentence_word[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
            sentence_pos[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
            sentence_deprel[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
            sentence_word[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
            sentence_pos[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
            sentence_deprel[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
           ]
        
    f.write(json.dumps(conf))
    
    
    while len(buffer) > 0:
        head_id_stack_right = sentence_head_id[stack[-1]] if len(stack) > 0 else -2
        head_id_buffer_left = sentence_head_id[buffer[0]]
        
        if (head_id_stack_right == buffer[0]):
            tree.append((stack[-1], sentence_deprel[stack[-1]], buffer[0])) # add stack[-1] to tree
            stack.pop(-1) # left arc
            f.write(" <> 0\n")
        
        elif (len(stack) > 0 and head_id_buffer_left == stack[-1] and \
              check_no_child(stack, buffer, buffer[0], sentence_head_id)):
            tree.append((buffer[0], sentence_deprel[buffer[0]], stack[-1])) # add buffer[0] to tree
            buffer[0] = stack.pop(-1) # right arc
            f.write(" <> 1\n")
            
        else:
            stack.append(buffer.pop(0)) # shift
            f.write(" <> 2\n")
        
        conf = [\
                sentence_word[stack[-1]] if len(stack)>0 else None, \
                sentence_pos[stack[-1]] if len(stack)>0 else None, \
                sentence_deprel[stack[-1]] if len(stack)>0 else None, \
                sentence_word[buffer[0]] if len(buffer)>0 else None, \
                sentence_pos[buffer[0]] if len(buffer)>0 else None, \
                sentence_deprel[buffer[0]] if len(buffer)>0 else None, \
                sentence_word[buffer[1]] if len(buffer)>1 else None, \
                sentence_pos[buffer[1]] if len(buffer)>1 else None, \
                sentence_deprel[buffer[1]] if len(buffer)>1 else None, \
                sentence_word[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
                sentence_pos[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
                sentence_deprel[ldep(stack, -1, tree)] if ldep(stack, -1, tree)!=None else None, \
                sentence_word[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
                sentence_pos[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
                sentence_deprel[rdep(stack, -1, tree)] if rdep(stack, -1, tree)!=None else None, \
                sentence_word[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
                sentence_pos[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
                sentence_deprel[ldep(buffer, 0, tree)] if ldep(buffer, 0, tree)!=None else None, \
                sentence_word[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
                sentence_pos[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
                sentence_deprel[rdep(buffer, 0, tree)] if rdep(buffer, 0, tree)!=None else None, \
               ]
        
        f.write(json.dumps(conf))
    f.write(" <> -1\n")
#     print(i)
print("Wrote ", filename)

Wrote  configurations_testing.txt


## Load pre-trained model

In [6]:
# from gensim.scripts import glove2word2vec

# glove2word2vec.glove2word2vec('glove.6B/glove.6B.50d.txt', 'glove.6B/glove.6B.50d.txt.word2vec')

In [7]:
model = gensim.models.KeyedVectors.load_word2vec_format('glove.6B/glove.6B.50d.txt.word2vec', binary=False)

## Load configuration training set

In [8]:
filename = "configurations_training.txt"
f = open(filename, "r")
lines = f.readlines()

embedded_input = []
tag_output = []
i = 0

for line in lines:
    i += 1
    line = line.split("<>")
    c = json.loads(line[0].strip())
    t = line[1].strip()
#     print(c)
    embedding = np.array(0)
    for elem in c:
        try:
            enc = model.wv[elem.lower()]
        except:
            enc = model.wv['null']
        if embedding.size == 1:
            embedding = np.append(embedding, enc)
            embedding = np.delete(embedding, 0)
        else:
            embedding = np.append(embedding, enc)
            
    embedded_input.append(embedding)
    tag_output.append(int(t))
    
    if i == 20000:
        break

print(len(embedded_input))
print(len(tag_output))

20000
20000


## Classifier

In [9]:
from sklearn.neural_network import MLPClassifier

In [10]:
clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 10), random_state=1)

In [11]:
clf.fit(embedded_input, tag_output)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [12]:
# prediction = clf.predict(embedded_input).tolist()
# actual_tag = tag_output
# # print(prediction)
# # print(actual_tag)

# c = 0
# for i in range(len(prediction)):
#     if prediction[i] == actual_tag[i]:
#         c += 1
# print(c*100/len(prediction))

## Accuracy calculation

### Load configuration test set

In [13]:
filename = "configurations_testing.txt"
f = open(filename, "r")
lines = f.readlines()

test_input = []
test_output = []
i = 0

for line in lines:
    i += 1
    line = line.split("<>")
    c = json.loads(line[0].strip())
    t = line[1].strip()
#     print(c)
    embedding = np.array(0)
    for elem in c:
        try:
            enc = model.wv[elem.lower()]
        except:
            enc = model.wv['null']
        if embedding.size == 1:
            embedding = np.append(embedding, enc)
            embedding = np.delete(embedding, 0)
        else:
            embedding = np.append(embedding, enc)
            
    test_input.append(embedding)
    test_output.append(int(t))
    
    if i == 8000:
        break

print(len(test_input))
print(len(test_output))

8000
8000


In [14]:
prediction = clf.predict(test_input).tolist()
actual_tag = test_output
# print(prediction)
# print(actual_tag)

c = 0
for i in range(len(prediction)):
    if prediction[i] == actual_tag[i]:
        c += 1
print("Accuracy: ", c*100/len(prediction))

Accuracy:  90.4125
